In [2]:
from utils import ParquetLoader, GPSSimplifier, GPSConverter

GPSConverter("b.gpx")
ParquetLoader().save(GPSSimplifier(30).simplify(ParquetLoader().load("b.parquet")), "b_simplified.parquet")

Converti b.gpx -> b.parquet
b.parquet chargé avec 2333 lignes.
b_simplified.parquet sauvegardé avec 210 lignes.


In [37]:
ParquetLoader().load('a.parquet')

a.parquet chargé avec 2168 lignes.


,latitude,longitude
0,48.33302,2.69233
1,48.33306,2.69237
2,48.33307,2.69238
3,48.33310,2.69238
4,48.33315,2.69236
...,...,...
2163,48.82680,2.35328
2164,48.82687,2.35325
2165,48.82756,2.35293
2166,48.82800,2.35273
